## Final version includes errors and inter-annual elevation, volume, and mass

### Filter out the original CSV file of the research area you need

In [15]:
import os
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import gc

# Step 2: Filter glaciers within study area bounds
def filter_glaciers_by_bounds(rgi_shapefile, bounds):
    """Filter glaciers based on bounding box"""
    minx, miny, maxx, maxy = bounds
    rgi = gpd.read_file(rgi_shapefile)
    study_area_glaciers = rgi.cx[minx:maxx, miny:maxy]
    return study_area_glaciers

# Step 2: Process large CSV file in chunks
def process_large_csv(input_csv, glacier_ids, output_csv, chunksize=500000):
    """Read large CSV file in chunks and filter by glacier IDs"""
    print(f"Starting processing: {input_csv}")
    
    if os.path.exists(output_csv):
        os.remove(output_csv)
    
    total_rows = sum(1 for _ in open(input_csv)) - 1
    total_chunks = total_rows // chunksize + 1
    
    with pd.read_csv(input_csv, chunksize=chunksize) as reader:
        for i, chunk in enumerate(tqdm(reader, total=total_chunks, desc="Processing chunks")):
            filtered_chunk = chunk[chunk['rgiid'].isin(glacier_ids)]
            
            if not filtered_chunk.empty:
                header = (i == 0)
                filtered_chunk.to_csv(output_csv, mode='a', header=header, index=False)
    
    print(f"✅ Data saved to: {output_csv}")

# Step 4: Aggregate by year at CSV level
def aggregate_csv_by_year(filtered_csv, output_csv, chunksize=100000):
    """Aggregate data by year at CSV level to avoid creating large GeoJSON files"""
    print("Aggregating CSV data directly...")
    
    # Initialize result DataFrame
    result_df = pd.DataFrame()
    
    # Read and aggregate CSV in chunks
    with pd.read_csv(filtered_csv, chunksize=chunksize) as reader:
        for chunk in tqdm(reader, desc="Aggregating CSV chunks"):
            # Convert time to year
            chunk['year'] = pd.to_datetime(chunk['time'], errors='coerce').dt.year
            chunk = chunk.dropna(subset=['year'])
            chunk['year'] = chunk['year'].astype(int)
            # Fields to aggregate
            agg_fields = ['dh', 'err_dh', 'area']
            group_fields = ['rgiid', 'year']
            ···
            # Perform aggregation
            chunk_agg = chunk.groupby(group_fields)[agg_fields].mean().reset_index()
            
            # Retain static fields (use first value)
            static_fields = ['RGIId', 'GLIMSId', 'Name', 'Zmin', 'Zmax', 'Zmed', 
                             'Lmax', 'Slope', 'Aspect', 'CenLat', 'CenLon']
            
            static_dict = {}
            for field in static_fields:
                if field in chunk.columns:
                    static_dict[field] = chunk.groupby('rgiid')[field].first()
            
            # Merge static fields
            for field, values in static_dict.items():
                chunk_agg[field] = chunk_agg['rgiid'].map(values)
            
            # Append to result
            if result_df.empty:
                result_df = chunk_agg
            else:
                result_df = pd.concat([result_df, chunk_agg], ignore_index=True)
            
            # Periodically aggregate and clean memory
            if len(result_df) > 500000:
                result_df = result_df.groupby(group_fields).agg({
                    **{field: 'mean' for field in agg_fields},
                    **{field: 'first' for field in static_fields if field in result_df.columns}
                }).reset_index()
                gc.collect()
    
    # Final aggregation
    final_df = result_df.groupby(group_fields).agg({
        **{field: 'mean' for field in agg_fields},
        **{field: 'first' for field in static_fields if field in result_df.columns}
    }).reset_index()
    
    # Save aggregated CSV
    final_df.to_csv(output_csv, index=False)
    print(f"✅ Aggregated CSV saved to: {output_csv}")
    
    return final_df

# Main function
def main():
    # Config paths
    STUDY_AREA_SHP = "influ_area/influ_in_TB/influ_in_TB.shp"
    RGI_SHP = "data/RGI/13_14_15_rgi60_HighMountainAsia/region_13_14_15_rgi60_HighMountainAsia.shp"
    INPUT_CSV = "data/theia_data/time_series_13/dh_13_14_15_rgi60_pergla_cumul.csv"
    FILTERED_CSV = "data/study_area_glacier_cumul_filtered.csv"
    AGGREGATED_CSV = "data/study_area_glacier_yearly_aggregated.csv"
    
    # Create output directories
    os.makedirs(os.path.dirname(FILTERED_CSV), exist_ok=True)
    os.makedirs(os.path.dirname(AGGREGATED_CSV), exist_ok=True)
    
    try:
        # Step 1: Get study area bounds
        gdf = gpd.read_file(STUDY_AREA_SHP)
        bounds = gdf.total_bounds
        print(f"Study area bounds: {bounds}")
        
        # Step 2: Filter glaciers within the study area
        study_area_glaciers = filter_glaciers_by_bounds(RGI_SHP, bounds)
        # Step 1.3
        glacier_ids = set(study_area_glaciers['RGIId'])
        print(f"Found {len(glacier_ids)} glaciers")
        
        # Step 3: Filter CSV data
        process_large_csv(INPUT_CSV, glacier_ids, FILTERED_CSV)
        
        # Step 4: Aggregate yearly at CSV level
        aggregated_df = aggregate_csv_by_year(FILTERED_CSV, AGGREGATED_CSV)
        
        # Output summary
        print("\n=== Processing completed ===")
        print(f"Total records: {len(aggregated_df)}")
        print(f"Year range: {aggregated_df['year'].min()} - {aggregated_df['year'].max()}")
        print(f"Number of glaciers: {aggregated_df['rgiid'].nunique()}")
        
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Study area bounds: [ 68.59166667  25.99164587 103.72083333  39.96248542]
Found 79953 glaciers
Starting processing: data/theia_data/time_series_13/dh_13_14_15_rgi60_pergla_cumul.csv


Processing chunks: 100%|██████████| 47/47 [06:29<00:00,  8.30s/it]


✅ Data saved to: data/study_area_glacier_cumul_filtered.csv
Aggregating CSV data directly...


Aggregating CSV chunks: 193it [02:00,  1.60it/s]


✅ Aggregated CSV saved to: data/study_area_glacier_yearly_aggregated.csv

=== Processing completed ===
Total records: 1679013
Year range: 2000 - 2020
Number of glaciers: 79953


### Split the original csv into 21 year csv (including errors)

In [12]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

def aggregate_and_calculate_yearly_change(input_csv, output_dir, chunksize=100000):
    """
    Aggregate CSV data by year and calculate yearly change (differential method)
    
    Parameters:
    input_csv: Path to the input CSV file
    output_dir: Path to the output directory
    chunksize: Chunk size for reading the CSV
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize dictionary to store glacier time series
    glacier_data = {}
    
    # Read and process CSV in chunks
    print("Reading and processing data...")
    with pd.read_csv(input_csv, chunksize=chunksize) as reader:
        for chunk in tqdm(reader, desc="Processing chunks"):
            # Convert time to year
            chunk['year'] = pd.to_datetime(chunk['time'], errors='coerce').dt.year
            chunk = chunk.dropna(subset=['year'])
            chunk['year'] = chunk['year'].astype(int)
            
            # Organize data by glacier
            for rgiid, group in chunk.groupby('rgiid'):
                if rgiid not in glacier_data:
                    glacier_data[rgiid] = {}
                
                # Aggregate by year (average within a year if multiple observations)
                for year, year_group in group.groupby('year'):
                    if year not in glacier_data[rgiid]:
                        glacier_data[rgiid][year] = {
                            'dh_cumulative': [],
                            'err_dh': [],
                            'area': []
                        }
                    
                    glacier_data[rgiid][year]['dh_cumulative'].extend(year_group['dh'].dropna().tolist())
                    glacier_data[rgiid][year]['err_dh'].extend(year_group['err_dh'].dropna().tolist())
                    glacier_data[rgiid][year]['area'].extend(year_group['area'].dropna().tolist())
    
    # Calculate yearly changes (differential method)
    print("Calculating yearly changes...")
    yearly_changes = []
    
    for rgiid, data in tqdm(glacier_data.items(), desc="Calculating yearly differentials"):
        years = sorted(data.keys())
        
        for i in range(len(years)):
            year = years[i]
            
            # Calculate average cumulative value for the year
            dh_cumulative_avg = np.mean(data[year]['dh_cumulative'])
            err_dh_avg = np.sqrt(np.mean(np.array(data[year]['err_dh'])**2))
            area_avg = np.mean(data[year]['area'])
            
            if i == 0:
                # First year: use cumulative value directly
                dh_yearly = dh_cumulative_avg
                err_dh_yearly = err_dh_avg
            else:
                # Later years: calculate difference from the previous year
                prev_year = years[i-1]
                if prev_year in data:
                    prev_dh_cumulative_avg = np.mean(data[prev_year]['dh_cumulative'])
                    prev_err_dh_avg = np.sqrt(np.mean(np.array(data[prev_year]['err_dh'])**2))
                    
                    # Calculate the differential
                    dh_yearly = dh_cumulative_avg - prev_dh_cumulative_avg
                    # Error propagation for the difference
                    err_dh_yearly = np.sqrt(err_dh_avg**2 + prev_err_dh_avg**2)
                else:
                    # Skip if previous year data is missing
                    continue
            
            yearly_changes.append({
                'rgiid': rgiid,
                'year': year,
                'dh_yearly': dh_yearly,
                'dh_cumulative': dh_cumulative_avg,
                'err_dh_yearly': err_dh_yearly,
                'err_dh_cumulative': err_dh_avg,
                'area': area_avg,
                'n_obs': len(data[year]['dh_cumulative'])
            })
    
    # Convert to DataFrame
    final_df = pd.DataFrame(yearly_changes)
    
    # Save CSV files by year
    years = sorted(final_df['year'].unique())
    for year in tqdm(years, desc="Saving yearly CSVs"):
        year_df = final_df[final_df['year'] == year].copy()
        output_path = os.path.join(output_dir, f'glacier_data_{year}.csv')
        year_df.to_csv(output_path, index=False)
        print(f"  - {year}: {len(year_df)} records -> {output_path}")
    
    # Print summary
    print(f"\n=== Processing complete ===")
    print(f"Total records: {len(final_df)}")
    print(f"Unique glaciers: {final_df['rgiid'].nunique()}")
    print(f"Year range: {final_df['year'].min()} - {final_df['year'].max()}")
    
    # Check yearly change statistics
    dh_yearly_stats = final_df['dh_yearly'].describe()
    print("\nYearly change statistics:")
    print(dh_yearly_stats)
    
    return final_df

if __name__ == "__main__":
    # Config
    INPUT_CSV = "data/study_area_glacier_cumul_filtered.csv"
    OUTPUT_DIR = "gee_output/yearly_data/yearly_csv_differential"
    
    # Execute processing
    aggregate_and_calculate_yearly_change(INPUT_CSV, OUTPUT_DIR)

Reading and processing data...


Processing chunks: 193it [11:00,  3.42s/it]


Calculating yearly changes...


Calculating yearly differentials:   0%|          | 0/79953 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.11/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
Saving yearly CSVs:   5%|▍         | 1/21 [00:00<00:14,  1.34it/s]

  - 2000: 79953 records -> gee_output/yearly_data/yearly_csv_differential/glacier_data_2000.csv


...

  - 2018: 79953 records -> gee_output/yearly_data/yearly_csv_differential/glacier_data_2018.csv


Saving yearly CSVs:  95%|█████████▌| 20/21 [00:16<00:00,  1.21it/s]

  - 2019: 79953 records -> gee_output/yearly_data/yearly_csv_differential/glacier_data_2019.csv


Saving yearly CSVs: 100%|██████████| 21/21 [00:16<00:00,  1.25it/s]

  - 2020: 79953 records -> gee_output/yearly_data/yearly_csv_differential/glacier_data_2020.csv

=== Processing complete ===
Total records: 1679013
Unique glaciers: 79953
Year range: 2000 - 2020

Yearly change statistics:
count    1.645308e+06
mean    -2.108579e-01
std      5.887732e-01
min     -3.641104e+01
25%     -4.862670e-01
50%     -1.702500e-01
75%      8.977564e-02
max      6.175394e+01
Name: dh_yearly, dtype: float64


### Calculate volume and mass (cumulative version)

In [3]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

def calculate_volume_mass_changes(csv_dir, output_dir):
    """
    Add volume and mass change columns to existing yearly CSVs
    
    Parameters:
    csv_dir: Input CSV directory
    output_dir: Output CSV directory
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Ice density constant (kg/m³)
    ICE_DENSITY = 850  # Can be adjusted based on the study area
    
    # Get all CSV files
    csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
    csv_files.sort()
    
    for csv_file in tqdm(csv_files, desc="Calculating volume and mass changes"):
        csv_path = os.path.join(csv_dir, csv_file)
        output_path = os.path.join(output_dir, csv_file)
        
        try:
            # Read CSV
            df = pd.read_csv(csv_path)
            
            # Calculate cumulative volume change (m³)
            # dV = dh_cumulative × area
            df['dV'] = df['dh_cumulative'] * df['area']
            
            # Calculate cumulative volume error (m³)
            # err_dV = err_dh_cumulative × area
            df['err_dV'] = df['err_dh_cumulative'] * df['area']
            
            # Calculate cumulative mass change (Gt)
            # dM = dV × density / 10^12 (to convert to Gt)
            df['dM'] = df['dV'] * ICE_DENSITY / 1e12
            
            # Calculate cumulative mass error (Gt)
            df['err_dM'] = df['err_dV'] * ICE_DENSITY / 1e12
            
            # Add additional statistical info
            # Note: This is now cumulative specific mass balance
            df['specific_mass_balance'] = df['dM'] / (df['area'] / 1e6)  # Gt/km²
            
            # Also calculate yearly changes if they exist
            if 'dh_yearly' in df.columns:
                df['dV_yearly'] = df['dh_yearly'] * df['area']
                df['dM_yearly'] = df['dV_yearly'] * ICE_DENSITY / 1e12
                df['specific_mass_balance_yearly'] = df['dM_yearly'] / (df['area'] / 1e6)
            
            # Save new CSV
            df.to_csv(output_path, index=False)
            
            # Output summary
            year = csv_file.split('_')[-1].replace('.csv', '')
            total_volume_change = df['dV'].sum()  # This is now cumulative
            total_mass_change = df['dM'].sum()    # This is now cumulative
            
            if 'dV_yearly' in df.columns:
                total_volume_change_yearly = df['dV_yearly'].sum()
                total_mass_change_yearly = df['dM_yearly'].sum()
                print(f"  - {year}: ")
                print(f"    Cumulative: Volume change = {total_volume_change/1e9:.3f} km³, Mass change = {total_mass_change:.3f} Gt")
                print(f"    Yearly: Volume change = {total_volume_change_yearly/1e9:.3f} km³, Mass change = {total_mass_change_yearly:.3f} Gt")
            else:
                print(f"  - {year}: Cumulative volume change = {total_volume_change/1e9:.3f} km³, Cumulative mass change = {total_mass_change:.3f} Gt")
            
        except Exception as e:
            print(f"Error processing {csv_file}: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"\n=== Volume and mass calculation completed ===")
    print(f"Output directory: {output_dir}")

if __name__ == "__main__":
    # Config
    INPUT_CSV_DIR = "gee_output/yearly_data/yearly_csv_differential"  # Input directory
    OUTPUT_CSV_DIR = "gee_output/yearly_data/yearly_csv_volume_mass"  # Output directory
    
    # Run calculation
    calculate_volume_mass_changes(INPUT_CSV_DIR, OUTPUT_CSV_DIR)

Calculating volume and mass changes:   5%|▍         | 1/21 [00:02<00:43,  2.19s/it]

  - 2000: 
    Cumulative: Volume change = 3.677 km³, Mass change = 3.126 Gt
    Yearly: Volume change = 3.677 km³, Mass change = 3.126 Gt


Calculating volume and mass changes:  10%|▉         | 2/21 [00:04<00:42,  2.23s/it]

  - 2001: 
    Cumulative: Volume change = -5.754 km³, Mass change = -4.891 Gt
    Yearly: Volume change = -9.432 km³, Mass change = -8.017 Gt


Calculating volume and mass changes:  14%|█▍        | 3/21 [00:06<00:40,  2.26s/it]

...

  - 2018: 
    Cumulative: Volume change = -351.228 km³, Mass change = -298.544 Gt
    Yearly: Volume change = -28.806 km³, Mass change = -24.485 Gt


Calculating volume and mass changes:  95%|█████████▌| 20/21 [00:44<00:02,  2.18s/it]

  - 2019: 
    Cumulative: Volume change = -380.373 km³, Mass change = -323.317 Gt
    Yearly: Volume change = -29.145 km³, Mass change = -24.774 Gt


Calculating volume and mass changes: 100%|██████████| 21/21 [00:46<00:00,  2.19s/it]

  - 2020: 
    Cumulative: Volume change = -406.118 km³, Mass change = -345.200 Gt
    Yearly: Volume change = -25.745 km³, Mass change = -21.883 Gt

=== Volume and mass calculation completed ===
Output directory: gee_output/yearly_data/yearly_csv_volume_mass


### Merge GeoJSON (cumulative version)

In [4]:
import os
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

def merge_csv_with_geometry(csv_dir, rgi_shapefile, study_area_shp, geojson_dir):
    """
    Merge CSV files with RGI geometry data.
    
    Parameters:
    csv_dir: Directory containing yearly CSV files (with volume and mass change results)
    rgi_shapefile: Path to the RGI shapefile
    study_area_shp: Path to the study area shapefile
    geojson_dir: Output directory for GeoJSON files
    """
    # Create output directory
    os.makedirs(geojson_dir, exist_ok=True)
    
    # Load study area bounds
    print("Loading study area bounds...")
    study_area_gdf = gpd.read_file(study_area_shp)
    study_area_gdf = study_area_gdf.to_crs(epsg=4326)
    bounds = study_area_gdf.total_bounds
    minx, miny, maxx, maxy = bounds
    print(f"Study area bounds: {bounds}")
    
    # Load RGI shapefile
    print("Loading RGI shapefile...")
    gdf_rgi = gpd.read_file(rgi_shapefile, bbox=(minx, miny, maxx, maxy))
    gdf_rgi = gdf_rgi.to_crs(epsg=4326)
    
    # Keep only necessary fields
    gdf_rgi = gdf_rgi[['RGIId', 'geometry', 'Area', 'Zmed', 'Slope']]
    print(f"Number of RGI records: {len(gdf_rgi)}")
    
    # List all CSV files
    csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
    csv_files.sort()
    
    # Process each CSV file
    for csv_file in tqdm(csv_files, desc="Merging geometry"):
        year = csv_file.split('_')[-1].replace('.csv', '')
        csv_path = os.path.join(csv_dir, csv_file)
        geojson_path = os.path.join(geojson_dir, f'glacier_data_{year}.geojson')
        
        try:
            # Read CSV
            df = pd.read_csv(csv_path)
            
            # Merge with geometry
            merged_gdf = df.merge(gdf_rgi, left_on='rgiid', right_on='RGIId', how='left')
            merged_gdf = gpd.GeoDataFrame(merged_gdf, geometry='geometry', crs="EPSG:4326")
            
            # Drop records without geometry
            merged_gdf = merged_gdf.dropna(subset=['geometry'])
            
            # Remove duplicate RGIId column if both exist
            if 'RGIId' in merged_gdf.columns and 'rgiid' in merged_gdf.columns:
                merged_gdf = merged_gdf.drop(columns=['RGIId'])
            
            # Save to GeoJSON
            merged_gdf.to_file(geojson_path, driver='GeoJSON')
            
            # Output statistics - now showing both cumulative and yearly changes if available
            print(f"✅ GeoJSON for {year} saved: {geojson_path}")
            print(f"   - Number of records: {len(merged_gdf)}")
            
            # Cumulative changes
            if 'dV' in merged_gdf.columns:
                print(f"   - Cumulative volume change: {merged_gdf['dV'].sum()/1e9:.3f} km³")
            if 'dM' in merged_gdf.columns:
                print(f"   - Cumulative mass change: {merged_gdf['dM'].sum():.3f} Gt")
            
            # Yearly changes (if available)
            if 'dV_yearly' in merged_gdf.columns:
                print(f"   - Yearly volume change: {merged_gdf['dV_yearly'].sum()/1e9:.3f} km³")
            if 'dM_yearly' in merged_gdf.columns:
                print(f"   - Yearly mass change: {merged_gdf['dM_yearly'].sum():.3f} Gt")
            
            # Average elevation changes
            if 'dh_cumulative' in merged_gdf.columns:
                print(f"   - Average cumulative elevation change: {merged_gdf['dh_cumulative'].mean():.3f} m")
            if 'dh_yearly' in merged_gdf.columns:
                print(f"   - Average yearly elevation change: {merged_gdf['dh_yearly'].mean():.3f} m")
            
        except Exception as e:
            print(f"Error processing year {year}: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"\n=== Geometry merging completed ===")
    print(f"GeoJSON output directory: {geojson_dir}")

if __name__ == "__main__":
    # Configuration
    CSV_DIR = "gee_output/yearly_data/yearly_csv_volume_mass"  # CSV directory containing volume and mass changes
    RGI_SHAPEFILE = "data/RGI/13_14_15_rgi60_HighMountainAsia/region_13_14_15_rgi60_HighMountainAsia.shp"
    STUDY_AREA_SHP = "influ_area/influ_in_TB/influ_in_TB.shp"
    GEOJSON_DIR = "gee_output/yearly_data/yearly_geojson_volume_mass"
    
    # Execute merging
    merge_csv_with_geometry(CSV_DIR, RGI_SHAPEFILE, STUDY_AREA_SHP, GEOJSON_DIR)

Loading study area bounds...
Study area bounds: [ 68.59166667  25.99164587 103.72083333  39.96248542]
Loading RGI shapefile...
Number of RGI records: 79953


Merging geometry:   5%|▍         | 1/21 [01:35<31:55, 95.75s/it]

✅ GeoJSON for 2000 saved: gee_output/yearly_data/yearly_geojson_volume_mass/glacier_data_2000.geojson
   - Number of records: 79953
   - Cumulative volume change: 3.677 km³
   - Cumulative mass change: 3.126 Gt
   - Yearly volume change: 3.677 km³
   - Yearly mass change: 3.126 Gt
   - Average cumulative elevation change: -0.008 m
   - Average yearly elevation change: -0.008 m


Merging geometry:  10%|▉         | 2/21 [03:03<28:44, 90.77s/it]

✅ GeoJSON for 2001 saved: gee_output/yearly_data/yearly_geojson_volume_mass/glacier_data_2001.geojson
   - Number of records: 79953
   - Cumulative volume change: -5.754 km³
   - Cumulative mass change: -4.891 Gt
   - Yearly volume change: -9.432 km³
   - Yearly mass change: -8.017 Gt
   - Average cumulative elevation change: -0.122 m
   - Average yearly elevation change: -0.114 m


...

✅ GeoJSON for 2018 saved: gee_output/yearly_data/yearly_geojson_volume_mass/glacier_data_2018.geojson
   - Number of records: 79953
   - Cumulative volume change: -351.228 km³
   - Cumulative mass change: -298.544 Gt
   - Yearly volume change: -28.806 km³
   - Yearly mass change: -24.485 Gt
   - Average cumulative elevation change: -3.887 m
   - Average yearly elevation change: -0.293 m


Merging geometry:  95%|█████████▌| 20/21 [27:59<01:21, 81.34s/it]

✅ GeoJSON for 2019 saved: gee_output/yearly_data/yearly_geojson_volume_mass/glacier_data_2019.geojson
   - Number of records: 79953
   - Cumulative volume change: -380.373 km³
   - Cumulative mass change: -323.317 Gt
   - Yearly volume change: -29.145 km³
   - Yearly mass change: -24.774 Gt
   - Average cumulative elevation change: -4.199 m
   - Average yearly elevation change: -0.312 m


Merging geometry: 100%|██████████| 21/21 [29:18<00:00, 83.76s/it]

✅ GeoJSON for 2020 saved: gee_output/yearly_data/yearly_geojson_volume_mass/glacier_data_2020.geojson
   - Number of records: 79953
   - Cumulative volume change: -406.118 km³
   - Cumulative mass change: -345.200 Gt
   - Yearly volume change: -25.745 km³
   - Yearly mass change: -21.883 Gt
   - Average cumulative elevation change: -4.428 m
   - Average yearly elevation change: -0.229 m

=== Geometry merging completed ===
GeoJSON output directory: gee_output/yearly_data/yearly_geojson_volume_mass


### Convert geojson to tiff (cumulative version)

In [5]:
import os
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm
import rasterio
from rasterio.transform import from_bounds
from rasterio.features import rasterize
import gc

def create_multiband_tiff_optimized(geojson_dir, study_area_shp, tiff_dir):
    """Optimized multi-band TIFF generation with cumulative changes"""
    # Create output directory
    os.makedirs(tiff_dir, exist_ok=True)
    
    # Load study area bounds
    print("Loading study area bounds...")
    study_area_gdf = gpd.read_file(study_area_shp)
    study_area_gdf = study_area_gdf.to_crs(epsg=4326)
    bounds = study_area_gdf.total_bounds
    minx, miny, maxx, maxy = bounds
    print(f"Study area bounds: {bounds}")
    
    # Set resolution (moderate resolution for faster processing)
    resolution_degrees = 0.001  # ~100 meters
    width = int((maxx - minx) / resolution_degrees)
    height = int((maxy - miny) / resolution_degrees)
    
    # Auto-adjust if grid size too large
    if width > 10000 or height > 10000:
        print(f"Original raster size too large ({width} x {height}), adjusting resolution...")
        resolution_degrees = max((maxx - minx) / 10000, (maxy - miny) / 10000)
        width = int((maxx - minx) / resolution_degrees)
        height = int((maxy - miny) / resolution_degrees)
    
    print(f"Raster settings: resolution={resolution_degrees:.6f} degrees, width={width}, height={height}")
    
    # Build transform matrix
    transform = from_bounds(minx, miny, maxx, maxy, width, height)
    
    # Process each GeoJSON
    geojson_files = [f for f in os.listdir(geojson_dir) if f.endswith('.geojson')]
    geojson_files.sort()
    
    for geojson_file in tqdm(geojson_files, desc="Generating 3-band TIFFs"):
        year = geojson_file.split('_')[-1].replace('.geojson', '')
        geojson_path = os.path.join(geojson_dir, geojson_file)
        tiff_path = os.path.join(tiff_dir, f'glacier_changes_{year}_3band.tif')
        
        try:
            print(f"  - Processing year {year}...")
            gdf = gpd.read_file(geojson_path)
            
            if gdf.crs is None or gdf.crs.to_epsg() != 4326:
                gdf = gdf.to_crs(epsg=4326)
            
            # Prepare data for rasterization - using cumulative changes
            shapes_dh_cumulative = []
            shapes_dV_cumulative = []
            shapes_dM_cumulative = []
            
            for idx, row in gdf.iterrows():
                if row['geometry'] is not None:
                    # Use cumulative elevation change
                    if not pd.isna(row['dh_cumulative']):
                        shapes_dh_cumulative.append((row['geometry'], float(row['dh_cumulative'])))
                    
                    # Use cumulative volume change
                    if not pd.isna(row['dV']):
                        shapes_dV_cumulative.append((row['geometry'], float(row['dV'] / 1e9)))  # km³
                    
                    # Use cumulative mass change
                    if not pd.isna(row['dM']):
                        shapes_dM_cumulative.append((row['geometry'], float(row['dM'])))
            
            # Rasterize all bands
            print(f"  - Rasterizing {len(shapes_dh_cumulative)} geometries...")
            dh_cumulative_raster = rasterize(
                shapes_dh_cumulative,
                out_shape=(height, width),
                transform=transform,
                fill=np.nan,
                all_touched=True,
                dtype=np.float32
            )
            
            dV_cumulative_raster = rasterize(
                shapes_dV_cumulative,
                out_shape=(height, width),
                transform=transform,
                fill=np.nan,
                all_touched=True,
                dtype=np.float32
            )
            
            dM_cumulative_raster = rasterize(
                shapes_dM_cumulative,
                out_shape=(height, width),
                transform=transform,
                fill=np.nan,
                all_touched=True,
                dtype=np.float32
            )
            
            # Save as multiband TIFF
            print(f"  - Saving TIFF...")
            with rasterio.open(
                tiff_path,
                'w',
                driver='GTiff',
                height=height,
                width=width,
                count=3,
                dtype=rasterio.float32,
                crs='EPSG:4326',
                transform=transform,
                nodata=np.nan,
                compress='lzw'
            ) as dst:
                dst.write(dh_cumulative_raster, 1)
                dst.write(dV_cumulative_raster, 2)
                dst.write(dM_cumulative_raster, 3)
                
                # Set band descriptions
                dst.set_band_description(1, 'cumulative_elevation_change_m')
                dst.set_band_description(2, 'cumulative_volume_change_km3')
                dst.set_band_description(3, 'cumulative_mass_change_Gt')
                
                # Add metadata
                dst.update_tags(
                    year=year,
                    description=f"Cumulative glacier changes by year {year}",
                    band1_units="meters",
                    band2_units="cubic kilometers",
                    band3_units="gigatonnes",
                    coordinate_system="WGS84",
                    epsg_code="4326"
                )
            
            print(f"✅ Saved 3-band TIFF for {year}: {tiff_path}")
            
            # Summary stats
            valid_pixels_dh = np.count_nonzero(~np.isnan(dh_cumulative_raster))
            mean_dh_cumulative = np.nanmean(dh_cumulative_raster)
            total_mass_change_cumulative = np.nansum(dM_cumulative_raster)
            total_volume_change_cumulative = np.nansum(dV_cumulative_raster)
            
            print(f"  - Valid pixels: {valid_pixels_dh}")
            print(f"  - Mean cumulative elevation change: {mean_dh_cumulative:.3f} m")
            print(f"  - Total cumulative volume change: {total_volume_change_cumulative:.3f} km³")
            print(f"  - Total cumulative mass change: {total_mass_change_cumulative:.3f} Gt")
            
            # Memory cleanup
            del gdf, dh_cumulative_raster, dV_cumulative_raster, dM_cumulative_raster
            del shapes_dh_cumulative, shapes_dV_cumulative, shapes_dM_cumulative
            gc.collect()
            
        except Exception as e:
            print(f"❌ Error processing year {year}: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"\n=== All 3-band TIFFs generated ===")
    print(f"Output directory: {tiff_dir}")

if __name__ == "__main__":
    # Config
    GEOJSON_DIR = "gee_output/yearly_data/yearly_geojson_volume_mass"
    STUDY_AREA_SHP = "influ_area/influ_in_TB/influ_in_TB.shp"
    TIFF_DIR = "gee_output/yearly_data/yearly_tiff_3band"
    
    # Run conversion
    create_multiband_tiff_optimized(GEOJSON_DIR, STUDY_AREA_SHP, TIFF_DIR)

Loading study area bounds...
Study area bounds: [ 68.59166667  25.99164587 103.72083333  39.96248542]
Original raster size too large (35129 x 13970), adjusting resolution...
Raster settings: resolution=0.003513 degrees, width=10000, height=3976


Generating 3-band TIFFs:   0%|          | 0/21 [00:00<?, ?it/s]

  - Processing year 2000...
  - Rasterizing 78348 geometries...
  - Saving TIFF...
✅ Saved 3-band TIFF for 2000: gee_output/yearly_data/yearly_tiff_3band/glacier_changes_2000_3band.tif


Generating 3-band TIFFs:   5%|▍         | 1/21 [01:24<28:11, 84.59s/it]

  - Valid pixels: 1136824
  - Mean cumulative elevation change: 0.034 m
  - Total cumulative volume change: 8065.706 km³
  - Total cumulative mass change: 6855.860 Gt
  - Processing year 2001...
  - Rasterizing 78348 geometries...
  - Saving TIFF...
✅ Saved 3-band TIFF for 2001: gee_output/yearly_data/yearly_tiff_3band/glacier_changes_2001_3band.tif


...

  - Valid pixels: 1136824
  - Mean cumulative elevation change: -4.071 m
  - Total cumulative volume change: -141375.656 km³
  - Total cumulative mass change: -120169.477 Gt
  - Processing year 2019...
  - Rasterizing 78348 geometries...
  - Saving TIFF...
✅ Saved 3-band TIFF for 2019: gee_output/yearly_data/yearly_tiff_3band/glacier_changes_2019_3band.tif


Generating 3-band TIFFs:  95%|█████████▌| 20/21 [28:42<01:25, 85.70s/it]

  - Valid pixels: 1136824
  - Mean cumulative elevation change: -4.411 m
  - Total cumulative volume change: -152903.422 km³
  - Total cumulative mass change: -129967.992 Gt
  - Processing year 2020...
  - Rasterizing 78348 geometries...
  - Saving TIFF...
✅ Saved 3-band TIFF for 2020: gee_output/yearly_data/yearly_tiff_3band/glacier_changes_2020_3band.tif


Generating 3-band TIFFs: 100%|██████████| 21/21 [30:08<00:00, 86.13s/it]

  - Valid pixels: 1136824
  - Mean cumulative elevation change: -4.699 m
  - Total cumulative volume change: -169641.875 km³
  - Total cumulative mass change: -144195.656 Gt

=== All 3-band TIFFs generated ===
Output directory: gee_output/yearly_data/yearly_tiff_3band
